In [ ]:
"""
Nim (3 piles, start at [5,5,5]) — Human vs Computer with fixed-depth (3) Minimax.

Rules (normal play):
- On your turn, remove 1..3 objects from exactly one pile that has enough objects.
- Game ends when no objects remain; the player who is to move (and thus cannot move) loses.

Run from terminal:
  python nim_minimax_depth3.py                 # Human vs Computer (Human starts)
  python nim_minimax_depth3.py --computer      # Computer starts
  python nim_minimax_depth3.py --simulate      # Computer vs Computer

Run inside Jupyter/Colab:
  - Simply import or execute this cell; extra notebook flags are ignored safely.
  - You can also call: simulate_ai_vs_ai() or play_human_vs_ai(human_starts=True)
"""

from __future__ import annotations
import argparse
import sys
from dataclasses import dataclass
from typing import List, Tuple, Optional

# ---------------- Configuration (assignment defaults) ----------------
MAX_REMOVE: int = 3
INITIAL_PILES: Tuple[int, int, int] = (5, 5, 5)
MAX_DEPTH: int = 3  # fixed by assignment

State = Tuple[int, int, int]
Move = Tuple[int, int]  # (pile_index, count_to_remove)

# ---------------- Core Game Logic ----------------
def is_terminal(state: State) -> bool:
    return sum(state) == 0

def legal_moves(state: State) -> List[Move]:
    moves: List[Move] = []
    for i, pile in enumerate(state):
        if pile <= 0:
            continue
        for k in range(1, min(MAX_REMOVE, pile) + 1):
            moves.append((i, k))
    return moves

def apply_move(state: State, move: Move) -> State:
    i, k = move
    if not (0 <= i < 3):
        raise ValueError("Invalid pile index; must be 0..2 (i.e., piles 1..3).")
    if not (1 <= k <= MAX_REMOVE):
        raise ValueError(f"You can remove between 1 and {MAX_REMOVE} objects.")
    piles = list(state)
    if piles[i] < k:
        raise ValueError("Cannot remove more objects than the pile has.")
    piles[i] -= k
    return (piles[0], piles[1], piles[2])

def nim_sum(state: State) -> int:
    from functools import reduce
    from operator import ixor
    return reduce(ixor, state, 0)

# ---------------- Evaluation (no ML) ----------------
def static_eval(state: State) -> int:
    """
    Heuristic from the perspective of the player TO MOVE at `state`.
      - If terminal: very bad (no move) -> large negative.
      - Bouton signal: +100 if nim-sum != 0, else -100.
      - Tie-breakers: subtract 10 * (#nonempty) and subtract total stones.
    """
    if is_terminal(state):
        return -10_000
    base = 100 if nim_sum(state) != 0 else -100
    nonempty = (1 if state[0] > 0 else 0) + (1 if state[1] > 0 else 0) + (1 if state[2] > 0 else 0)
    total = state[0] + state[1] + state[2]
    return base - 10 * nonempty - total

@dataclass
class MinimaxResult:
    value: int
    move: Optional[Move]

# ---------------- Minimax (fixed depth = 3) ----------------
def minimax(state: State, depth: int, maximizing: bool) -> MinimaxResult:
    if depth == 0 or is_terminal(state):
        return MinimaxResult(static_eval(state), None)

    moves = legal_moves(state)
    if not moves:
        return MinimaxResult(-10_000, None)

    # Optional deterministic ordering: try moves that make nim-sum 0 and reduce total quickly
    def move_key(m: Move):
        s2 = apply_move(state, m)
        return (0 if nim_sum(s2) == 0 else 1, -(s2[0] + s2[1] + s2[2]))
    moves.sort(key=move_key)

    if maximizing:
        best_val = -10**9
        best_move: Optional[Move] = None
        for m in moves:
            s2 = apply_move(state, m)
            res = minimax(s2, depth - 1, maximizing=False)
            if res.value > best_val or (res.value == best_val and best_move is None):
                best_val, best_move = res.value, m
        return MinimaxResult(best_val, best_move)
    else:
        best_val = 10**9
        best_move = None
        for m in moves:
            s2 = apply_move(state, m)
            res = minimax(s2, depth - 1, maximizing=True)
            if res.value < best_val or (res.value == best_val and best_move is None):
                best_val, best_move = res.value, m
        return MinimaxResult(best_val, best_move)

# ---------------- I/O Helpers ----------------
def format_state(state: State) -> str:
    return f"[{state[0]}, {state[1]}, {state[2]}]"

def ask_human_move(state: State) -> Move:
    while True:
        try:
            raw = input("Your move (pile index 1-3 and count 1-3, e.g. '2 3'): ").strip()
        except EOFError:
            print("\nEOF received; exiting.")
            sys.exit(0)
        except KeyboardInterrupt:
            print("\nInterrupted; exiting.")
            sys.exit(0)

        if raw.lower() in {"q", "quit", "exit"}:
            print("Exiting game. Bye!")
            sys.exit(0)

        parts = raw.split()
        if len(parts) != 2:
            print("Please enter exactly two numbers, e.g.: 2 1")
            continue

        try:
            i = int(parts[0]) - 1
            k = int(parts[1])
        except ValueError:
            print("Both entries must be integers, e.g.: 3 2")
            continue

        if not (0 <= i < 3):
            print("Pile index must be in 1..3")
            continue
        if not (1 <= k <= MAX_REMOVE):
            print(f"You must remove 1..{MAX_REMOVE} objects.")
            continue
        if state[i] < k:
            print(f"Pile {i+1} has only {state[i]} objects.")
            continue

        return (i, k)

def computer_move(state: State) -> Move:
    res = minimax(state, MAX_DEPTH, maximizing=True)
    if res.move is None:
        raise RuntimeError("Computer has no legal move.")
    return res.move

# ---------------- Game Modes ----------------
def play_human_vs_ai(human_starts: bool = True) -> None:
    print("=== Nim: Human vs Computer (Minimax depth=3) ===")
    print(f"Initial piles: {format_state(INITIAL_PILES)}")
    state = INITIAL_PILES
    human_turn = human_starts

    while True:
        if is_terminal(state):
            if human_turn:
                print("All piles empty. You cannot move. You LOSE.")
                print(">> Computer WINS!")
            else:
                print("All piles empty. Computer cannot move. Computer LOSES.")
                print(">> You WIN!")
            break

        if human_turn:
            print(f"\nCurrent piles: {format_state(state)}")
            mv = ask_human_move(state)
            state = apply_move(state, mv)
            print(f"You removed {mv[1]} from pile {mv[0]+1} -> {format_state(state)}")
        else:
            print(f"\nComputer thinking (depth {MAX_DEPTH})...")
            mv = computer_move(state)
            state = apply_move(state, mv)
            print(f"Computer removed {mv[1]} from pile {mv[0]+1} -> {format_state(state)}")

        human_turn = not human_turn

def simulate_ai_vs_ai() -> None:
    print("=== Nim: Simulation (Computer vs Computer, depth=3) ===")
    print(f"Initial piles: {format_state(INITIAL_PILES)}")
    state = INITIAL_PILES
    turn = 0  # even -> A, odd -> B

    while True:
        if is_terminal(state):
            loser = "Computer A" if turn % 2 == 0 else "Computer B"
            winner = "Computer B" if loser == "Computer A" else "Computer A"
            print(f"All piles empty. {loser} cannot move. {winner} WINS!")
            break

        mv = computer_move(state)
        state = apply_move(state, mv)
        player = "Computer A" if turn % 2 == 0 else "Computer B"
        print(f"{player} removed {mv[1]} from pile {mv[0]+1} -> {format_state(state)}")
        turn += 1

# ---------------- Entry Point ----------------
def parse_args(argv: List[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser(description="Nim with fixed-depth (3) Minimax.")
    group = parser.add_mutually_exclusive_group()
    group.add_argument("--simulate", "-s", action="store_true",
                       help="Run Computer vs Computer simulation.")
    group.add_argument("--computer", "-c", action="store_true",
                       help="Computer starts (Human vs Computer).")
    parser.add_argument("--depth", type=int, default=MAX_DEPTH,
                        help="(Fixed at 3 by assignment; changing has no effect.)")

    # Use parse_known_args so notebook/Colab injected flags (e.g., -f <kernel.json>) are ignored
    args, unknown = parser.parse_known_args(argv)
    if unknown:
        # Keep output quiet in notebooks; uncomment for debugging:
        # print(f"[Info] Ignoring unknown args: {unknown}")
        pass
    return args

def main(argv: Optional[List[str]] = None) -> None:
    if argv is None:
        argv = sys.argv[1:]
    args = parse_args(argv)

    # Enforce fixed depth regardless of flag value (assignment requirement)
    if args.depth != MAX_DEPTH:
        print(f"[Info] Depth is fixed by assignment to {MAX_DEPTH}; ignoring --depth={args.depth}.")

    if args.simulate:
        simulate_ai_vs_ai()
    else:
        play_human_vs_ai(human_starts=not args.computer)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nInterrupted. Goodbye!")


=== Nim: Human vs Computer (Minimax depth=3) ===
Initial piles: [5, 5, 5]

Current piles: [5, 5, 5]
Your move (pile index 1-3 and count 1-3, e.g. '2 3'): 1 2
You removed 2 from pile 1 -> [3, 5, 5]

Computer thinking (depth 3)...
Computer removed 3 from pile 1 -> [0, 5, 5]

Current piles: [0, 5, 5]
Your move (pile index 1-3 and count 1-3, e.g. '2 3'):  1 2
Pile 1 has only 0 objects.
Your move (pile index 1-3 and count 1-3, e.g. '2 3'): 2 1
You removed 1 from pile 2 -> [0, 4, 5]

Computer thinking (depth 3)...
Computer removed 1 from pile 3 -> [0, 4, 4]

Current piles: [0, 4, 4]
Your move (pile index 1-3 and count 1-3, e.g. '2 3'): 2 1
You removed 1 from pile 2 -> [0, 3, 4]

Computer thinking (depth 3)...
Computer removed 2 from pile 2 -> [0, 1, 4]

Current piles: [0, 1, 4]
Your move (pile index 1-3 and count 1-3, e.g. '2 3'): 3 2
You removed 2 from pile 3 -> [0, 1, 2]

Computer thinking (depth 3)...
Computer removed 1 from pile 3 -> [0, 1, 1]

Current piles: [0, 1, 1]
Your move (pile in

# How to execute
* simulate AI vs AI  
simulate_ai_vs_ai()  
* or start human vs computer (human starts)  
play_human_vs_ai(human_starts=True)

# Summary

Decision rule 1 (terminal): if [0,0,0], the side to move loses.

Decision rule 2 (nim-sum test):

If A xor B xor C = 0 ⇒ losing position (optimal opponent can mirror).

Else ⇒ winning position — there exists a move to make nim-sum 0.

Decision rule 3 (optimal move with 1–3 bound):

Let S = A xor B xor C. Choose a pile Pi where (Pi xor S) < Pi, set Δ = Pi - (Pi xor S).

If 1 ≤ Δ ≤ 3, remove Δ from that pile (reach nim-sum 0).

If Δ > 3 (bound prevents ideal move), use a bounded heuristic (take up to 3 from the largest pile) — still legal, keeps the expert system consistent with assignment constraints.
